## 講習2 --- IRAFで1次処理

IRAFのタスクを使って、**１次処理**(生データからダークを引き、それをフラットで割り、スカイバイアスを引く) を行ってみましょう。  

ここでは、IRAFの基本タスク、**imarith**, **imcombine**を使います。  

通常、近赤外線観測ではそのあとにregistration(複数枚のイメージの位置合わせ)をしてスタックするという作業までして1次処理なのですが、ここでは時間の関係上省略します。 

### サンプルデータ

sample_data2　のディレクトリの中には、サンプルの生データセットがあります。  
南アフリカIRSF望遠鏡のSIRIUSのHバンドのデータです。  
FITSファイルの名前は  **ir[4桁のフレーム番号].fits** となっています。  (SIRIUSのオリジナルのファイル名から変更しています)

| フレーム番号| オブジェクト | 積分時間   (1枚あたり) |  
| :-: | :- | :- |
|0001-0005 |  dome_flat_off  | 5秒 |
|0006-0010 | dome_flat_on  | 5秒|
|0011-0019 | target1  | 5秒 |   
|0020-0028 | target2  | 10秒  | 
|0029-0033 | dark  | 5秒  |
|0034-0038 | dark  | 10秒 |

フラット補正用のデータを、ドームフラットをランプをONとOFFでそれぞれ5枚1セットで取得しています。  
天体については2天体をそれぞれディザリングして9枚づつ取得しています。  
最後にダークフレームを5秒と10秒をそれぞれ5枚づつ取得しています。(ダークフレームは、天体の観測に使用した積分時間に対応するものが必要です。)


### target1の1次処理  

ここでは、target1を対象として、1次処理を行ってみます。  
ここでの工程は
- ダーク補正用フレームの作成  -> (1) とする
- フラット補正用フレームの作成 -> (2) とする
- ターゲットの生データから(1)を引き、それを(2)で割る -> (3)とする
- (3) の集合からスカイバイアスフレームを作成する (セルフスカイ) -> (4)とする
- (3) のそれぞれから(4)を引く

通常は「ダーク補正用フレーム」のことを単純に「ダーク」、「フラット補正用フレーム」のことを「フラット」と省略して呼びます。


#### ダーク補正用フレームの作成

「生データからダークを引き」のダークは、通常、複数枚(10枚程度)のダークの平均(メジアン)から作成するダーク補正用フレームを指します。  
平均(メジアン)をとることで、もっともらしいダーク補正用フレームを作成します。S/Nの良いフレームとなり、平均(メジアン)をとることで、偶然読み出しが変なフレームが混ざっていても、その影響を無しに、あるいは小さくすることができます。  

#### iraf.imcombine 

irafのimcombineがまさに、複数枚のフレームの平均を作成するためのタスクです。  


In [1]:
from pyraf import iraf

In [2]:
iraf.imcombine.unlearn()

ファイル名をカンマ区切りで並べた文字列をinputとして与えます。するとこれらのファイルをスタックして平均を計算してくれます。  

下では文字列が長いので、バックスラッシュ\ を使って折り返します。バックスラッシュの後にスペースを入れないこと。
(macではOption + ¥でバックスラッシュになります)

In [3]:
iraf.imcombine.input = 'sample_data2/ir0029.fits, sample_data2/ir0030.fits, \
    sample_data2/ir0031.fits, sample_data2/ir0032.fits, sample_data2/ir0033.fits'

出力先ファイル名をdark5.fitsとし、medianで平均をとります。

In [4]:
iraf.imcombine(output='dark5.fits', combine='median')  


Jul 26  9:45: IMCOMBINE
  combine = median, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  sample_data2/ir0029.fits
  sample_data2/ir0030.fits
  sample_data2/ir0031.fits
  sample_data2/ir0032.fits
  sample_data2/ir0033.fits

  Output image = dark5.fits, ncombine = 5


#### _補足_  
'sample_data2/ir0029.fits, sample_data2/ir0030.fits, sample_data2/ir0031.fits, sample_data2/ir0032.fits, sample_data2/ir0033.fits' って長いですよね。ここでは5つのファイルだけでしたが、もっと長くなることだってありえます。  
下のように、globを利用して、こんなふうに該当する文字列を作成することができます。

In [5]:
import glob
comstr = '' 
for item in glob.glob('sample_data2/*.fits'):
    if 29 <= int(item[-7:-5])  <= 33:  #  4桁の番号の最後の二桁
        comstr += item + ',' 

In [6]:
comstr[:-1]  # 末尾の , を削除

'sample_data2/ir0029.fits,sample_data2/ir0030.fits,sample_data2/ir0031.fits,sample_data2/ir0032.fits,sample_data2/ir0033.fits'

#### フラット補正用フレームの作成  

フラット補正用フレームの作成方法には何通りかありますが、ここではドームフラットを使います。  
(IRSF/SIRIUSでは通常、トワイライトフラットを使うのですが、ここでは例外的に、処理工程のより簡単なドームフラットを使います。)  

工程は次の4ステップです。  
- ランプOFFのドームフラットの平均フレーム作成 ->(1)とする
- ランプONのドームフラットの平均フレーム作成 ->(2)とする
- (2) から (1)を引く -> (3)とする
- (3) を規格化する 

In [7]:
iraf.imcombine.input = 'sample_data2/ir0001.fits, sample_data2/ir0002.fits, \
    sample_data2/ir0003.fits, sample_data2/ir0004.fits, sample_data2/ir0005.fits'

In [8]:
iraf.imcombine(output='domeoff.fits', combine='median')  


Jul 26  9:46: IMCOMBINE
  combine = median, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  sample_data2/ir0001.fits
  sample_data2/ir0002.fits
  sample_data2/ir0003.fits
  sample_data2/ir0004.fits
  sample_data2/ir0005.fits

  Output image = domeoff.fits, ncombine = 5


In [9]:
iraf.imcombine.input = 'sample_data2/ir0006.fits, sample_data2/ir0007.fits, \
    sample_data2/ir0008.fits, sample_data2/ir0009.fits, sample_data2/ir0010.fits'

In [10]:
iraf.imcombine(output='domeon.fits', combine='median')  


Jul 26  9:46: IMCOMBINE
  combine = median, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  sample_data2/ir0006.fits
  sample_data2/ir0007.fits
  sample_data2/ir0008.fits
  sample_data2/ir0009.fits
  sample_data2/ir0010.fits

  Output image = domeon.fits, ncombine = 5


domeoff.fitsとdomeon.fitsができました。この二つの画像の引き算をするために、irafの**imarith**を使います。

In [11]:
iraf.imarith('domeon.fits', '-', 'domeoff.fits', 'flat.fits')

次に、flat.fitsを規格化します。具体的には、メジアン値で画像全体を割ります。

In [12]:
iraf.imstat.unlearn()
iraf.imstat.fields = 'midpt, stddev'
iraf.imstat.lower = 100
iraf.imstat.upper = 10000
iraf.imstat.nclip = 5

In [13]:
iraf.imstat('flat.fits')

#     MIDPT    STDDEV
      3412.     132.2


In [14]:
iraf.imarith('flat.fits', '/', 3412, 'flat.fits')  #  元のファイルへの上書きはOK 

既存のファイル名のファイルに書き出そうとするとエラーがでます。元のファイルへの上書き更新はOKです。

#### ターゲットからダークを引き、フラットで割る  

target1の観測セットの1番目のフレームについて、ダーク引き、フラット割りの処理をしてみます。

In [15]:
iraf.imarith('sample_data2/ir0011.fits', '-', 'dark5.fits', 'tmp.fits')
iraf.imarith('tmp.fits', '/', 'flat.fits', 'tmp.fits')

ir0011.fits からdarkを引き、フラットで割った結果のファイル名をidf0011.fitsのように、先頭についている文字列をirからidf(darkとflat処理済み)と変えた名前にしてやり、通し番号は同じにしておくことにします。


``` python 
iraf.imarith('sample_data2/ir0011.fits', '-', 'dark5.fits', 'idf0011.fits')
iraf.imarith('idf0011.fits', '/', 'flat.fits', 'idf0011.fits')

iraf.imarith('sample_data2/ir0012.fits', '-', 'dark5.fits', 'idf0012.fits')
iraf.imarith('idf0012.fits', '/', 'flat.fits', 'idf0012.fits')

iraf.imarith('sample_data2/ir0013.fits', '-', 'dark5.fits', 'idf0013.fits')
iraf.imarith('idf0013.fits', '/', 'flat.fits', 'idf0013.fits')
```
... 
と真面目に全部書くのは面倒ですね。  
これを、0011から0019の全てについてまとめて行うには、次のようにforループで行います。

In [16]:
for num in range(11, 20): #  注意！ 19ではなく20！
    snum = str(num) #  数値を文字列に変換
    rfile = 'sample_data2/ir00' + snum + '.fits'
    iraf.imarith(rfile, '-', 'dark5.fits', 'idf00' + snum + '.fits') 
    iraf.imarith('idf00' + snum + '.fits', '/', 'flat.fits', 'idf00' + snum + '.fits')

通し番号だけをforループの繰り返し変数として扱うほうが、下のよりも簡単にプログラムが書けます。
``` python 
for rfile in ['sample_data2/ir0011.fits', 'sample_data2/ir0012.fits',  ... (以下省略)]:
    ... (以下省略) 
```

#### _補足_  
上の場合だと、「9から13まで」とか「98から102まで」のように、数値の桁数が変わる場合には困りますね。
通し番号をより一般的に扱うには、**zfill()**を使います。引数で指定した桁数の数の文字列を作成します。

In [17]:
x = str(10)
x.zfill(4)

'0010'

書き換えると下のようになります。
``` python 
for num in range(11, 20): 
    snum = str(num).zfill(4) #  数値を文字列に変換してから、4桁の文字列に
    rfile = 'sample_data2/ir' + snum + '.fits'
    iraf.imarith(rfile, '-', 'dark5.fits', 'idf' + snum + '.fits') 
    iraf.imarith('idf' + snum + '.fits', '/', 'flat.fits', 'idf' + snum + '.fits')
``` 

#### スカイバイアスを作成  

ここでは、セルフスカイを作成して、それをスカイバイアスとします。  
視野に写っている星の数が少ない、星雲状の天体が写っていない、といった場合に、自らのディザリングのセットのフレームのメジアンの画像を作成することで、星が消えて、「足し算成分」のスカイバイアス画像が得られます。


In [18]:
iraf.imcombine.input = 'idf0011.fits, idf0012.fits, idf0013.fits, idf0014.fits, idf0015.fits, \
    idf0016.fits, idf0017.fits, idf0018.fits, idf0019.fits'

**[ポイント!]**  iraf.imcombine.zero = 'median' としてやることで、星がよりキレイに消えます。 combine時にバックグラウンドの値を、足したり引いたりして、そろえることになります。それにより、外れ値がちゃんと除外されます。

In [20]:
iraf.imcombine(output='skybias1.fits', zero='median', combine='median')  


Jul 26  9:47: IMCOMBINE
  combine = median, scale = none, zero = median, weight = none
  blank = 0.
                Images   Median    Zero
           idf0011.fits  1991.5      0.
           idf0012.fits  1976.6  14.882
           idf0013.fits  1971.7  19.846
           idf0014.fits  1983.3   8.224
           idf0015.fits  1984.7  6.8293
           idf0016.fits  1994.4 -2.8705
           idf0017.fits  2004.2 -12.681
           idf0018.fits  1993.6 -2.1014
           idf0019.fits  1976.5  15.052

  Output image = skybias1.fits, ncombine = 9


#### スカイバイアスを引く 

ダーク引き、フラット割りをした画像からスカイバイアスを引きます。  

In [21]:
for num in range(11, 20): 
    snum = str(num) #  数値を文字列に変換
    iraf.imarith('idf00' + snum + '.fits', '-', 'skybias1.fits', 'idfs00' + snum + '.fits')

### 演習2 
target2を対象として、上の1次処理を行ってください。  
ただし、積分時間が10秒なので、10秒のダーク補正用フレームを作成する必要があります。  
フラット補正用フレームは上で作成したもの(flat.fits)が使えます。  (sample_data3にも入っています。)

**ボーナス問題**  
zero='median'を指定せずに、スカイバイアスを作成してみましょう。  
そして、そのスカイバイアスをidf0020.fitsから引いてみましょう。 (このとき書き出すファイル名を変えておきましょう)
zero='median'を指定したものと比較してみてください。